In [ ]:
# === Confluent Cloud API key: lkc-ny81p3 ===

# API key:
# YIRFSMXHBDYQ276E

# API secret:
# 0emmkb0JWtDvxjJkkxsBa6CrchZN1TcHW4TRk0EBOQkO6uFuNJJrsmOV3LOpwxcn

# Bootstrap server:
# pkc-6vz38.westus2.azure.confluent.cloud:9092


In [5]:
def read_ccloud_config(config_file):
    conf = {}
    with open(config_file) as fh:
        for line in fh:
            line = line.strip()
            if len(line) != 0 and line[0] != "#":
                parameter, value = line.strip().split('=', 1)
                conf[parameter] = value.strip()
    return conf

In [14]:
from confluent_kafka import Producer
import json
# value ={"flight_num": "AA489", "from":"JFK", "to":"LAX", "old_departure_time": "2023-09-17T23:00:00"", 
#         "old_arrival_time":"2023-09-18T01:00:00","departure_time": "2020-09-18T02:00:00", 
#         "arrival_time":"2023-09-18T04:00:00", "reason":"Forecasted bad weather at destination",
#         "flight_status": "delayed"}
value ={"flight_num": "AA489", "from":"JFK", "to":"LAX", "old_departure_time": "NA", "old_arrival_time":"NA","departure_time": "2023-09-17T23:00:00", "arrival_time":"2023-09-18T01:00:00", 
        "reason":"NA","flight_status": "ontime"}

value = json.dumps(value)
producer = Producer(read_ccloud_config("client.properties"))
producer.produce("my-topic", key="", value=value)


In [9]:
from confluent_kafka import Consumer

props = read_ccloud_config("client.properties")
props["group.id"] = "python-group-1"
props["auto.offset.reset"] = "earliest"

consumer = Consumer(props)
consumer.subscribe(["my-topic"])
msg = consumer.poll(1.0)
if msg is not None and msg.error() is None:
     print("key = {key:12} value = {value:12}".format(key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))
     print(json.loads(msg.value().decode('utf-8')))

consumer.close()

In [ ]:
import ast
text ='{"Offset":13,"Partition":4,"Topic":"my-topic","Timestamp":"2023-09-17T23:57:50.248Z","Value":"{\"flight_id\": \"AA-102\", \"flight_time\": \"2020-01-01T00:00:00Z\", \"flight_status\": \"on-time\"}","Key":"key2","Headers":[]}'
json.loads(text)

In [ ]:
import json
a = json.loads('{"flight_num": "AA-102", "from":"SEATAC", "to":"SFO", "departure_time": "2020-01-01T00:00:00Z", "arrival_time":"2020-01-01T01:00:00Z", "flight_status": "on-time"}')


In [ ]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.models import Vector 
import openai
AZURE_OPENAI_ENDPOINT="https://openai002.openai.azure.com/"
AZURE_OPENAI_API_KEY="db77747b3587407496f968ac23008831"
AZURE_OPENAI_EMB_DEPLOYMENT="text-embedding-ada-002"
AZURE_OPENAI_CHAT_DEPLOYMENT="gpt-4"
USE_AZCS="False"
AZURE_SEARCH_SERVICE_ENDPOINT="https://cogsearch001.search.windows.net"
AZURE_SEARCH_INDEX_NAME="flights"
CACHE_INDEX_NAME="techdocscache"
AZURE_SEARCH_ADMIN_KEY="2HwNw2JAmUjp8lYFfiiIMeIFTPye8rHHq9ZYu3hc9YAzSeA9Vl5L"
AZURE_OPENAI_API_VERSION="2023-07-01-preview"
USE_SEMANTIC_CACHE="False"
SEMANTIC_HIT_THRESHOLD=0.9

credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
azcs_search_client = SearchClient(AZURE_SEARCH_SERVICE_ENDPOINT, index_name =AZURE_SEARCH_INDEX_NAME , credential=credential)
def generate_embeddings(text):
    print("emb_engine", AZURE_OPENAI_EMB_DEPLOYMENT)
    openai.api_version = "2023-05-15"
    response = openai.Embedding.create(
        input=text, engine=AZURE_OPENAI_EMB_DEPLOYMENT)
    embeddings = response['data'][0]['embedding']
    return embeddings

def search_knowledgebase(search_query):
    vector = Vector(value=generate_embeddings(search_query), k=3, fields="embedding")
    print("search query: ", search_query)
    results = azcs_search_client.search(  
        search_text=search_query,  
        vectors= [vector],
        select=["sourcepage","content"],
        top=5
    )  
    text_content =""
    for result in results:  
        text_content += f"{result['sourcepage']}\n{result['content']}\n"
    print("text_content", text_content)
    return text_content

def add_or_update(new_flight):
    azcs_search_client.merge_or_upload_documents(documents = [new_flight])

new_flight ={"flight_num": "AA-101", "from": "SEATAC", "to": "HOU", "departure_time": "2020-01-01T00:00:00Z", "arrival_time": "2020-01-01T01:00:00Z", "flight_status": "on-time"}

In [ ]:
new_flight['departure_time']

In [ ]:
add_or_update(new_flight)

In [ ]:
data ="""
{"Offset":53,"Partition":4,"Topic":"my-topic","Timestamp":"2023-09-18T03:27:44.863Z","Value":"{\"flight_num\": \"AA-101\", \"from\": \"SEATAC\", \"to\": \"HOU\", \"departure_time\": \"2020-01-01T00:00:00Z\", \"arrival_time\": \"2020-01-01T01:00:00Z\", \"flight_status\": \"on-time\"}","Key":"key2","Headers":[]}
2023-09-18T03:27:45Z   [Information]   {'Headers': '[]', 'Partition': '4', 'Value': '{"flight_num": "AA-101", "from": "SEATAC", "to": "HOU", "departure_time": "2020-01-01T00:00:00Z", "arrival_time": "2020-01-01T01:00:00Z", "flight_status": "on-time"}', 'Timestamp': '09/18/2023 03:27:44', 'Offset': '53', 'Key': 'key2', 'Topic': 'my-topic'}
"""
json.loads(data)

In [31]:
# Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
from pathlib import Path  
from dotenv import load_dotenv

env_path = Path('..') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
openai.api_key =  os.environ.get("AZURE_OPENAI_API_KEY")
openai.api_base =  os.environ.get("AZURE_OPENAI_ENDPOINT")


# openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_version = "2023-07-01-preview"
openai.api_type = "azure"
# openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")

messages = [{'role': 'system', 'content': "\nYou are Maya, an airline customer agent helping customers with questions and requests about their flight.\nYou are currently serving John Doe with id 12345.\nFirst, you need to look up their flight information and confirm with the customer about their flight information including flight number, from and to, departure and arrival time.\nWhen you are asked with a question knowlege question such as about their baggage limit, use the search tool to find relavent knowlege articles to create the answer.\nAnswer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the information from the search. If asking a clarifying question to the user would help, ask the question.\nIf the user is asking for information that is not related to flight and airline, say it's not your area of expertise.\nWhen the user is asking for a flight status, you can use the flight status API to check the flight status.\nWhen the user is asking to change their flight, first check the feasibility and consequence of the flight change, then if feasiable present information for confirmation from the customer. When customer agrees, execute the change.\n"}, {'role': 'assistant', 'content': 'Hi John Doe, this is Maya, your airline customer specialist, what can I do for you?'}, {'role': 'user', 'content': "hi, what's my flight detail?"}, {'role': 'assistant', 'name': 'load_user_flight_info', 'content': '{\n  "user_id": "12345"\n}'}, {'role': 'function', 'name': 'load_user_flight_info', 'content': "{'customer_id': '12345', 'name': 'John Doe', 'airline': 'Delta', 'flight_num': 'DL123', 'seat_num': '1A', 'departure_airport': 'JFK', 'arrival_airport': 'LAX', 'departure_time': '2020-01-0T00:00:00Z', 'arrival_time': '2020-01-01T02:15:00Z', 'ticket_class': 'First Class', 'ticket_num': '1234567890', 'gate': 'A1', 'boarding_time': '2019-31-12T23:15:00Z', 'boarding_zone': '1', 'boarding_group': 'A', 'boarding_position': '1', 'boarding_sequence': '1', 'boarding_pass_barcode': 'https://www.qrstuff.com/images/sample.png'}"}, {'role': 'assistant', 'content': 'Your ticket is with Delta for flight DL123. Your seat number is 1A in First Class. \n\nThe flight is departing from JFK airport and arriving at LAX airport. The departure time is on 2020-01-01 at 00:00 UTC, and the arrival time is expected to be at 02:15 on the same day.\n\nYour gate is A1, and the boarding starts at 23:15 on 2019-12-31. As a First Class passenger, you are in boarding zone 1, group A, position 1. Please make sure to be at the gate on time.\n\nYour ticket number is 1234567890. Please let me know if you need further assistance or have any other questions.'}, {'role': 'user', 'content': "I'd like to fly earlier, what options do I have?"}, {'role': 'assistant', 'name': 'query_flights', 'content': '{\n  "from_": "JFK",\n  "to": "LAX",\n  "departure_time": "2019-12-31"\n}'}, {'role': 'function', 'name': 'query_flights', 'content': 'flight_num AA3550, from: JFK, to: LAX, departure_time: 2020-09-20T10:30:00, arrival_time: 2020-12-20T10:30:00, flight_status: on time \nflight_num AA8984, from: JFK, to: LAX, departure_time: 2020-09-20T10:30:00, arrival_time: 2020-12-20T10:30:00, flight_status: on time \nflight_num AA1216, from: JFK, to: LAX, departure_time: 2020-09-20T10:30:00, arrival_time: 2020-12-20T10:30:00, flight_status: on time \n'}, {'role': 'assistant', 'content': 'I found three flights departing from JFK to LAX on 2019-12-31:\n\n1. Flight AA3550 departing at 10:30\n2. Flight AA8984 departing at 10:30\n3. Flight AA1216 departing at 10:30\n\nThey are all with the American Airlines, and currently on time. \n\nPlease note that if you change your flight, there may be a possible fee depending on the ticket type and the proximity of the new flight. Also, the availability of First Class may vary.\n\nWould you like to proceed with changing to one of these flights? If so, please specify which flight number.\n'}, {'role': 'user', 'content': 'AA1216  pls'}, {'role': 'user', 'content': 'option 3 please'}]

FUNCTIONS_SPEC= [  
    {
        "name": "search_knowledgebase",
        "description": "Searches the knowledge base for an answer to the question",
        "parameters": {
            "type": "object",
            "properties": {
                "search_query": {
                    "type": "string",
                    "description": "The search query to use to search the knowledge base"
                }
            },
            "required": ["search_query"],
        },

    },
    {
        "name": "query_flights",
        "description": "Query the list of available flights for a given departure airport code, arrival airport code and departure time",
        "parameters": {
            "type": "object",
            "properties": {
                "from_": {
                    "type": "string",
                    "description": "The departure airport code"
                },
                "to": {
                    "type": "string",
                    "description": "The arrival airport code"
                },
                "departure_time": {
                    "type": "string",
                    "description": "The departure time"
                }
            },
            "required": ["from_", "to", "departure_time"],
        },
    
    },
    {
        "name": "check_change_flight",
        "description": "Check the feasibility and outcome of a presumed flight change",
        "parameters": {
            "type": "object",
            "properties": {
                "flight_num": {
                    "type": "string",
                    "description": "The flight number"
                },
                "from_": {
                    "type": "string",
                    "description": "The departure airport code"
                },
                "to": {
                    "type": "string",
                    "description": "The arrival airport code"
                },
                "departure_time": {
                    "type": "string",
                    "description": "The departure time"
                }
            },
            "required": ["flight_num", "from_", "to", "departure_time"],
        },

    },

    {
        "name": "confirm_change_flight",
        "description": "Confirm change the flight for a customer",
        "parameters": {
            "type": "object",
            "properties": {
                "flight_num": {
                    "type": "string",
                    "description": "The flight number"
                },
                "from_": {
                    "type": "string",
                    "description": "The departure airport code"
                },
                "to": {
                    "type": "string",
                    "description": "The arrival airport code"
                },
                "departure_time": {
                    "type": "string",
                    "description": "The departure time"
                }
            },
            "required": ["flight_num", "from_", "to", "departure_time"],
        },

    },
    {
        "name": "check_flight_status",
        "description": "Check the flight status for a flight",
        "parameters": {
            "type": "object",
            "properties": {

                "flight_num": {
                    "type": "string",
                    "description": "The flight number"
                },
                "from_": {
                    "type": "string",
                    "description": "The departure airport code"
                }
            },
            "required": ["flight_num", "from_"],
        },
    },
    {
        "name": "load_user_flight_info",
        "description": "Loads the flight information for a user",
        "parameters": {
            "type": "object",

            "properties": {
                "user_id": {

                    "type": "string",
                    "description": "The user id"
                }
            },
            "required": ["user_id"],
        },
    }

]  
response = openai.ChatCompletion.create(
    engine="gpt-4",
    messages=messages,
    functions=FUNCTIONS_SPEC,
    function_call="auto", 
)

print(response['choices'][0]['message'])


{
  "role": "assistant",
  "function_call": {
    "name": "check_change_flight",
    "arguments": "{\n  \"flight_num\": \"AA1216\",\n  \"from_\": \"JFK\",\n  \"to\": \"LAX\",\n  \"departure_time\": \"2019-12-31T10:30:00\"\n}"
  }
}


In [33]:
import os
import sys
import datetime
import openai
import dotenv
import streamlit as st
from pathlib import Path
import requests
from audio_recorder_streamlit import audio_recorder
import json
# import API key from .env file
env_path = Path('.') / 'secrets.env'
dotenv.load_dotenv(dotenv_path=env_path)
openai.api_key =  os.environ.get("AZURE_OPENAI_API_KEY")
openai.api_base =  os.environ.get("AZURE_OPENAI_ENDPOINT")
openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"
chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")

whisper_engine = os.getenv("WHISPER_DEPLOYMENT")
# print(whisper_engine)

def create_invoice(**kwargs):
    pass
PERSONA = """
You are an accountant helping to create entry into the accounting system for a new invoice.
You listen to the audio recording of the invoice and enter the information into the accounting system.
From the audio recording, you hear the following information about the invoice's header
- Invoice number
- Invoice date
- Invoice amount
- Vendor name
- Vendor address
From the audio recording, you hear the following information about the invoice's line items
- Line item number
- Line item description
- Line item amount
- Line item quantity
- Line item price
- Line item tax
Then you enter the information into the accounting system using create_invoice function

"""

AVAILABLE_FUNCTIONS = {
            "create_invoice": create_invoice,

        } 

FUNCTIONS_SPEC= [  
    {
        "name": "create_invoice",
        "description": "Create an invoice in the accounting system",
        "parameters": {
            "type": "object",
            "properties": {
            "invoice_number": { "type": "string", "description": "The invoice number" },
            "invoice_date": { "type": "string", "description": "The invoice date" },
            "invoice_amount": { "type": "string", "description": "The invoice amount" },
            "vendor_name": { "type": "string", "description": "The vendor name" },
            "vendor_address": { "type": "string", "description": "The vendor address" },
            "invoice_line_items": { "type": "array", "description": "The invoice line items",
                    "items": {  
                        "line_item_number": { "type": "string", "description": "The line item number" },
                        "line_item_description": { "type": "string", "description": "The line item description" },
                        "line_item_amount": { "type": "string", "description": "The line item amount" },
                        "line_item_quantity": { "type": "string", "description": "The line item quantity" },
                        "line_item_price": { "type": "string", "description": "The line item price" },
                        "line_item_tax": { "type": "string", "description": "The line item tax" },

                    },
            },


        },

    },
    }

]  

def invoice_entry(persona,functions_spec, transcript):
        
        conversation =[{"role":"system", "content":persona}, {"role":"user", "content":transcript}]

        response = openai.ChatCompletion.create(
            deployment_id=chat_deployment, # The deployment name you chose when you deployed the GPT-35-turbo or GPT-4 model.
            messages=conversation,
        functions=functions_spec,
        function_call="auto", 
        )
        response_message = response["choices"][0]["message"]

        output =""
            # Step 2: check if GPT wanted to call a function
        if  response_message.get("function_call"):
            print("Recommended Function call:")
            print(response_message.get("function_call"))
            print()
            
            # Step 3: call the function
            # Note: the JSON response may not always be valid; be sure to handle errors
            
            function_name = response_message["function_call"]["name"]
            print("function_name", function_name)
            # verify function has correct number of arguments
            function_args = json.loads(response_message["function_call"]["arguments"])
            
            print("function_args", function_args)
            output= f"Recommended Function call: {function_name} with arguments {function_args}"
        return output

transcript_text= """ this is the invoice information: 

- Invoice number: 1233
- Invoice date:12/12/2012
- Invoice amount: 200
- Vendor name: IBM
- Vendor address: 1234 Main St, Seattle, WA 98101
- Line item number: 1
- Line item description: laptop
- Line item amount: 100
- Line item quantity: 2
- Line item price: 50
- Line item tax: 10
- Line item number: 2
- Line item description: mouse
- Line item amount: 100
- Line item quantity: 2
- Line item price: 50
- Line item tax: 10

"""

invoice_entry(PERSONA, FUNCTIONS_SPEC, transcript_text)



Recommended Function call:
{
  "name": "create_invoice",
  "arguments": "{\n  \"invoice_number\": \"1233\",\n  \"invoice_date\": \"12/12/2012\",\n  \"invoice_amount\": \"200\",\n  \"vendor_name\": \"IBM\",\n  \"vendor_address\": \"1234 Main St, Seattle, WA 98101\",\n  \"invoice_line_items\": [\n    {\n      \"line_item_number\": \"1\",\n      \"line_item_description\": \"laptop\",\n      \"line_item_amount\": \"100\",\n      \"line_item_quantity\": \"2\",\n      \"line_item_price\": \"50\",\n      \"line_item_tax\": \"10\"\n    },\n    {\n      \"line_item_number\": \"2\",\n      \"line_item_description\": \"mouse\",\n      \"line_item_amount\": \"100\",\n      \"line_item_quantity\": \"2\",\n      \"line_item_price\": \"50\",\n      \"line_item_tax\": \"10\"\n    }\n  ]\n}"
}

function_name create_invoice
function_args {'invoice_number': '1233', 'invoice_date': '12/12/2012', 'invoice_amount': '200', 'vendor_name': 'IBM', 'vendor_address': '1234 Main St, Seattle, WA 98101', 'invoice_li

"Recommended Function call: create_invoice with arguments {'invoice_number': '1233', 'invoice_date': '12/12/2012', 'invoice_amount': '200', 'vendor_name': 'IBM', 'vendor_address': '1234 Main St, Seattle, WA 98101', 'invoice_line_items': [{'line_item_number': '1', 'line_item_description': 'laptop', 'line_item_amount': '100', 'line_item_quantity': '2', 'line_item_price': '50', 'line_item_tax': '10'}, {'line_item_number': '2', 'line_item_description': 'mouse', 'line_item_amount': '100', 'line_item_quantity': '2', 'line_item_price': '50', 'line_item_tax': '10'}]}"